In [ ]:
# import os
# import shutil
# # # # 指定要删除的目录路径
# directory_path = '/kaggle/working/ws_model'
#
# # 检查目录是否存在
# if os.path.exists(directory_path) and os.path.isdir(directory_path):
#     try:
#         # 递归删除非空目录
#         shutil.rmtree(directory_path)
#         print(f'{directory_path} 已成功删除。')
#     except OSError as e:
#         print(f'删除 {directory_path} 时出错: {e}')
# else:
#     print(f'{directory_path} 不存在或者不是目录。')

In [ ]:
# ! git init
# ! git clone https://github.com/HEIHEILIN1/ws_model.git


In [ ]:
# import sys
# import torch.optim as optim
# from tqdm import tqdm
import torch.optim as optim
#
# sys.path.append("/kaggle/input/yolo-lib/YOLO")
#
# print(sys.executable)  # 查看 Python 解释器路径
# print(sys.path)       # 查看模块搜索路径

In [ ]:
# import os
# yolo_path = "/kaggle/input/yolo-lib/YOLO/ultralytics/cfg/models/v8"
# print(os.listdir(yolo_path))  # 查看YOLO目录内容
# # 检查关键文件是否存在
# print(os.path.exists(f"{yolo_path}/ultralytics/__init__.py"))  # 必须为True

In [ ]:
import ultralytics

from ultralytics.data.split import autosplit

from ultralytics.utils.loss import v8DetectionLoss

In [ ]:
# input_dir="/kaggle/working/ws_model/"
# k_fold_result_dir="/kaggle/working/ws_model/data/datasets/waste_classification"
# working_dir='/kaggle/working/ws_model/runs/detect'

In [ ]:
def configure_optimizer(model, lr=1e-4, lr_multiplier=10, weight_decay=0.05):
    """配置分层学习率优化器"""
    pretrained_params = []  # 骨干网络
    attention_params = []   # SEAttention模块
    head_params = []        # 检测头
    
    for name, param in model.named_parameters():
        if "SEAttention" in name:
            attention_params.append(param)
        elif "backbone" in name:
            pretrained_params.append(param)
        else:
            head_params.append(param)
    
    return optim.AdamW([
        {"params": pretrained_params, "lr": lr},
        {"params": attention_params, "lr": lr * (lr_multiplier ** 0.5)},
        {"params": head_params, "lr": lr * lr_multiplier}
    ], weight_decay=weight_decay)

In [ ]:
def train_with_coslr(model, data_yaml, config_path, pretrained_weights, epochs=100):
    """分阶段训练（带余弦退火调度）"""
    # 阶段1：冻结骨干
    print("🚀 阶段1：冻结骨干，训练注意力+检测头")
    for name, param in model.model.named_parameters():
        if "backbone" in name and "SEAttention" not in name:
            param.requires_grad = False
    
    # 阶段1训练
    model.train(
        cfg=config_path,
        data=data_yaml,
        epochs=1,
        lr0=1e-3,
        name="SEAtt_train/phase1"
    )
    
    # 阶段2：解冻骨干
    print("🔍 阶段2：解冻骨干，完整训练")
    for param in model.model.parameters():
        param.requires_grad = True
    
    # 配置优化器
    optimizer = configure_optimizer(
        model.model,
        lr=1e-4,  # 固定初始学习率
        lr_multiplier=10,
        weight_decay=0.05
    )
    
    # 配置调度器
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=7,
        T_mult=2,
        eta_min=1e-6
    )
    
    # 关键修改：创建自定义Trainer类
    class CustomTrainer(model.trainer.__class__):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.scheduler = None
        
        def set_optimizer(self, optimizer, scheduler):
            self.optimizer = optimizer
            self.scheduler = scheduler
        
        def on_train_epoch_end(self):
            if self.scheduler:
                self.scheduler.step()
            super().on_train_epoch_end()
    
    # 阶段2训练
    print("🚀 阶段2：使用固定学习率策略训练")
    
    # 获取默认训练参数
    args = {
        "cfg": config_path,
        "data": data_yaml,
        "epochs": epochs-5,
        "name": "SEAtt_train/phase2",
        "optimizer": "AdamW"  # 必须指定一个有效名称
    }
    
    # 创建自定义Trainer实例
    trainer = CustomTrainer(overrides=args)
    trainer.set_optimizer(optimizer, scheduler)
    trainer.model = model.model
    model.trainer = trainer
    
    # 开始训练
    model.trainer.train()

    

In [ ]:

autosplit('data/datasets/waste_classification/images', weights=(0.99, 0.01, 0),
              annotated_only=True)

# 配置参数
config_path = 'cfg/train.yaml'  # 训练配置文件路径
data_yaml = 'data/datasets/waste_classification/waste_classification.yaml'  # 数据配置文件路径
pretrained_weights = 'weights/yolov8n_pretrained.pt'  # 预训练权重路径

# 加载模型
model = ultralytics.YOLO("cfg/models/v8/yolov8-se-att.yaml").load(pretrained_weights)

# 开始训练
train_with_coslr(
        model=model,
        data_yaml=data_yaml,
        config_path=config_path,
        pretrained_weights=pretrained_weights,
        epochs=100
    )

阶段	训练策略	学习率设置	目的
阶段 1	冻结骨干	仅优化注意力 + 检测头	让新模块快速适应任务
阶段 2	解冻骨干	分层学习率（骨干低，注意力中，检测头高）	微调整个模型
监控	检查梯度	调整学习率乘数	确保注意力模块有效学习


In [ ]:
# !git pull https://github.com/HEIHEILIN1/ws_model.git